In [15]:
import gradio
from dotenv import load_dotenv
from pypdf import PdfReader
from openai import OpenAI

In [16]:
load_dotenv()

openai = OpenAI()

In [17]:
reader = PdfReader("me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text
    
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [18]:
name = "Zalaid Butt"

In [19]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so. Only reply precisly whatever user ask only reply precisly dont give anyother information"

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [20]:
from pydantic import BaseModel

class EvaluationSchema(BaseModel):
    approve: bool
    feedback: str


evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."


In [21]:
def evaluator(message,response) -> EvaluationSchema:
    response = openai.beta.chat.completions.parse(
    messages = [{'role': 'system','content': evaluator_system_prompt}] + [{'role':'user','content':f'this is the user msg: {message} and this is the agent reply: {response}'}],
    model = "gpt-4o-mini",
    response_format=EvaluationSchema)
    return response.choices[0].message.parsed



In [22]:
def rerun(response,message,feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{response}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-3.5-turbo", messages=messages)
    return response.choices[0].message.content

In [27]:

def chat(message,history):
    messages = [{'role':'system','content': system_prompt + "BE CONCISE TO THE QS DONT GIVE DETAIL IF NOT ASKED"}] + history + [{'role':'user','content':message}]
    response = openai.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = messages
    )
    response = response.choices[0].message.content

    check = evaluator(message,response)

    if check.approve:
        print("Passed evaluation - returning reply")
        return response
    else:
        print("failed going to rerun")
        print(response)
        final = rerun(response,message,check.feedback)
        return final






In [28]:
gradio.ChatInterface(chat,type='messages').launch()

* Running on local URL:  http://127.0.0.1:7882
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply


In [ ]:
gradio.ChatInterface(chat,type='messages').launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
failed going to rerun


: 

: 